# Mount Drive/smm4h

In [0]:
from google.colab import drive
GOOGLE_DRIVE_MOUNT = "/content/gdrive"
drive.mount(GOOGLE_DRIVE_MOUNT)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%cd ./gdrive/My\ Drive/smm4h

/content/gdrive/My Drive/smm4h


In [0]:
! git remote set-url origin https://<username>:<password>github.com/toskn/nru_hse_team_hlpl2020.git
! git pull origin master --allow-unrelated-histories
! git push --set-upstream origin master

/bin/bash: username: No such file or directory
From https://github.com/toskn/nru_hse_team_hlpl2020
 * branch            master     -> FETCH_HEAD
Already up to date.


KeyboardInterrupt: ignored

# Read and preparate data

In [0]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfVectorizer


DATA_FOLDER = 'original_data/'
data_train = pd.read_csv(DATA_FOLDER + 'task2_ru_training.tsv', sep='\t')
data_val = pd.read_csv(DATA_FOLDER + 'task2_ru_validation.tsv', sep='\t')

Train = data_train[['tweet', 'class']]
Train.fillna('', inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [0]:
Train['class'].value_counts()

0    5557
1     533
Name: class, dtype: int64

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

tf_idf = TfidfVectorizer()
tf_idf.fit(Train['tweet'])

# with open("tfidf.pkl", 'wb') as handle:
#     pickle.dump(tf_idf, handle)

X_train_tf_idf = tf_idf.transform(Train['tweet'])
y_train = Train['class']

In [0]:
Val = data_val[['tweet', 'class']]
Val.fillna('', inplace=True)

X_val = Val[['tweet']]
X_val_tf_idf = tf_idf.transform(X_val['tweet'])
y_val = Val['class']

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


---------------------

In [0]:
from sklearn.metrics import roc_auc_score

def eval_model(X, X_prepared, y_true, model, verbose=True):
    y_prob = model.predict_proba(X_prepared)[:,1]
    y_prob = pd.Series(data=y_prob, index=y_true.index)
    
    av_auc = 0
    for cat in X.category.unique():
        if len(y_true[indexes].unique()) == 1:
            continue
        cat_auc = roc_auc_score(y_true[indexes], y_prob[indexes])
        av_auc += cat_auc
        if verbose:
            print((cat + "(" + str(len(y_true[indexes])) + ") :").ljust(40), cat_auc.round(5))
    av_auc /= len(X.category.unique())
    if verbose:
        print()
        print("average roc_auc by all categories:", av_auc.round(5))
    return av_auc

# Models

## LightGBM

In [0]:
%%time

from lightgbm import LGBMClassifier

lgbmodel = LGBMClassifier(
    boosting_type="gbdt",
    objective="binary",
    # num_leaves=30,
    # max_depth=10,
    # n_estimators=800,
#     min_child_samples=100,
#     max_bin=100,
#     subsample=0.7,
#     subsample_freq=1,
#     colsample_bytree=0.9,
#     min_child_weight=0,
#     scale_pos_weight=200
)

lgbmodel = lgbmodel.fit(X_train_tf_idf, 
                        y_train, 
#                         eval_set=[(X_train_tf_idf, y_train),(X_test_tf_idf, y_test)],
                        eval_metric='auc'
                        )

CPU times: user 2.35 s, sys: 26.8 ms, total: 2.37 s
Wall time: 1.26 s


In [0]:
y_prob = lgbmodel.predict_proba(X_val_tf_idf)[:,1]
y_pred = lgbmodel.predict(X_val_tf_idf)
f1_score(y_val, y_pred)

0.29545454545454547

# LogReg

In [0]:
from sklearn.linear_model import LogisticRegression

log_regr = LogisticRegression()
log_regr.fit(X_train_tf_idf, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_prob = log_regr.predict_proba(X_val_tf_idf)[:,1]
y_pred = log_regr.predict(X_val_tf_idf)
print(roc_auc_score(y_val, y_prob))
print(f1_score(y_val, y_pred))

0.8291787784796766
0.07092198581560283


In [0]:
len(y_val[y_val == 1])

133

# LinearSVC

In [0]:
from sklearn.svm import LinearSVC

svc = LinearSVC(random_state=0, dual=False)

svc.fit(X_train_tf_idf, y_train)

y_pred = svc.predict(X_val_tf_idf)
f1_score(y_val, y_pred)

0.28415300546448086

# SVC

In [0]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='perceptron', # hinge, log, modified_huber, squared_hinge, perceptron
                    # learning_rate='optimal', # 'adaptive', 'invscaling', 'optimal'
                    penalty='l2',
                    tol=0.0007,
                    max_iter=100000,
                    random_state=101
                    )
clf.fit(X_train_tf_idf, y_train)

y_pred = clf.predict(X_val_tf_idf)
f1_score(y_val, y_pred)

0.3693379790940767

# CatBoost

In [0]:
! pip install catboost

     |████████████████████████████████| 64.4MB 49kB/s 


In [0]:
from catboost import CatBoostClassifier, Pool, cv

cat = CatBoostClassifier(
    # custom_loss=['Accuracy'],
    # random_seed=42,
    # logging_level='Silent',
    iterations=10000,
    # max_depth=4,
    task_type='GPU',
    verbose=0
)


cat.fit(
    X_train_tf_idf, y_train
);

In [0]:
y_prob = cat.predict_proba(X_val_tf_idf)[:,1]
y_pred = cat.predict(X_val_tf_idf)
f1_score(y_val, y_pred)

0.20224719101123595

0.20224719101123595

------------------------